In [1]:
import json
import pandas as pd
import numpy as np
import os 
from os import listdir
from os.path import isfile, join
from datetime import datetime

In [2]:
dir='C:\\Users\hecto\PreWorks\Ironhack_labs\Slack_FP\Slack/'
fol = os.listdir(dir)
print(fol)

['berlin-get-together', 'channels.json', 'data-memes', 'data-resources', 'data-viz-beauties', 'general', 'getting_started_tips', 'integration_logs.json', 'katas', 'lab-questions', 'music', 'project-questions', 'random', 'room4', 'users.json']


In [3]:
dchannels=[file for file in fol if file.endswith('json')]
channels=list(set(fol)-set(dchannels))
channels

['music',
 'berlin-get-together',
 'random',
 'data-memes',
 'data-resources',
 'katas',
 'project-questions',
 'lab-questions',
 'room4',
 'data-viz-beauties',
 'general',
 'getting_started_tips']

In [30]:
date_time = []
user = []
text = []
file_content = []
reactions = []
reactions_user=[]
replies_user = []
reply_users=[]
channel=[]
replies_time=[]
have_link=[]
link=[]
file_url=[]
have_file=[]
for c in channels:
    files = [f for f in listdir("slack/" +c) if isfile(join("slack/" +c, f))]

    for file in files:
        data=json.load(open( "slack/"+c +"/" + file,encoding='utf-8'))
        count=0
        for message in data:
            channel.append(c)
            date_time.append(message["ts"])
            text.append(message['text'])
            user.append(message["user"])
            file_content.append("files" in message.keys())
            
            if 'files' in message.keys():
                file_url.append(message['files'][0]['url_private_download'])
                have_file.append('True')
            else:
                file_url.append(0)
                have_file.append('False')
            if 'attachments' in message.keys():
                if 'title_link' in message['attachments'][0].keys():
                    link.append(message['attachments'][0]['title_link'])
                    have_link.append('True')
                else:
                    link.append(0)
                    have_link.append('False')
            else:
                link.append(0)
                have_link.append('False')
            if 'reply_users_count' in message.keys():
                reply_users.append(message['reply_users_count'])
            else: 
                reply_users.append(0)
                    
            if 'replies' in message.keys():
                replies_time.append(message['replies'][0]['ts'])
                replies_user.append(message['replies'][0]['user'])
            else: 
                replies_time.append(0)
                replies_user.append(0)
            
            if 'reactions' in message.keys():
                count=0
                for reaction in message["reactions"]:
                    count=count+reaction['count']
                reactions.append(count)
            else: 
                reactions.append(0)
            if 'reactions' in message.keys():
                reactions_user.append(message['reactions'][0]['users'][0])
            else:
                reactions_user.append(0)

In [5]:
df = pd.DataFrame({"daytime":date_time,
                        "user": user,'text':text,'file_content':file_content,'reactions':reactions,
                        'replies_time':replies_time,'channel':channel,'reply_users':reply_users,'replies_user':replies_user,
                        'have_file':have_file,'have_link':have_link,
                        'reactions_user':reactions_user})

In [34]:
data

[{'client_msg_id': 'b8b84807-8af8-4f2d-aadd-d071fd996460',
  'type': 'message',
  'text': 'yep, thanks!',
  'user': 'U021ZC6L676',
  'ts': '1622530807.012700',
  'team': 'T021KE79B9B',
  'user_team': 'T021KE79B9B',
  'source_team': 'T021KE79B9B',
  'user_profile': {'avatar_hash': 'b2575327f328',
   'image_72': 'https://avatars.slack-edge.com/2021-05-15/2092238648800_b2575327f3280aef4bbd_72.jpg',
   'first_name': 'Uroš',
   'real_name': 'Uroš Petković',
   'display_name': 'Uroš Petković',
   'team': 'T021KE79B9B',
   'name': 'urospet',
   'is_restricted': False,
   'is_ultra_restricted': False},
  'blocks': [{'type': 'rich_text',
    'block_id': 'JQjp',
    'elements': [{'type': 'rich_text_section',
      'elements': [{'type': 'text', 'text': 'yep, thanks!'}]}]}],
  'thread_ts': '1622475999.010700',
  'parent_user_id': 'U021ZC6L676'},
 {'client_msg_id': 'bddc7881-6895-4998-984b-3838f7dda837',
  'type': 'message',
  'text': "i'm getting this error - Cloning into 'JupiterLab'...\nfatal: r

In [6]:
df.shape

(1827, 12)

In [7]:

df['daytime']=pd.to_numeric(df['daytime'])
df['replies_time']=pd.to_numeric(df['replies_time'])
df["daytime"] = df["daytime"].apply(lambda x: round(x))
df["replies_time"] = df["replies_time"].apply(lambda x: round(x))
def time_conversor(x):
    return datetime.utcfromtimestamp(x).strftime('%d-%m-%Y %H:%M')
df["dayt"] = df["daytime"].apply(lambda x: time_conversor(x))
df["rep_time"] = df["replies_time"].apply(lambda x: time_conversor(x))
df["dayt"]=df["dayt"].apply(lambda x: datetime.strptime(x,'%d-%m-%Y %H:%M'))
df['date'] = pd.to_datetime(df['dayt']).dt.date
df['time'] = pd.to_datetime(df['dayt']).dt.time
df['rep_time']=df['rep_time'].astype(str)
#df_final["dayt"]=df_final["dayt"].apply(lambda x: datetime.strptime(x,'%H:%M'))
df['rep_time'] = pd.to_datetime(df['rep_time']).dt.time
df.drop(['dayt','daytime','replies_time'], axis=1, inplace=True)

In [8]:
users=json.load(open('slack/users.json'))
user_id=[]
name=[]
for user in users:
    user_id.append(user['id'])
    if 'real_name' in user.keys():
        name.append(user['real_name'])
    else:
        name.append(0)

users=dict(zip(user_id,name))

In [9]:
id_list=list(df['user'])
df['user']=[users[item] for item in id_list]
rea_list=list(df['reactions_user']) 
for i, item in enumerate(rea_list):
    if item != 0:
        rea_list[i]=users[item]
df['reactions_user']=rea_list
rep_list=list(df['replies_user'])
for x, item in enumerate(rep_list):
    if item!=0:
        rep_list[x]=users[item]
df['replies_user']=rep_list

In [10]:
import seaborn as sns
from textblob import TextBlob 
import matplotlib as plt
import docx2txt
import nltk
from nltk.tokenize import word_tokenize
from nltk.text import Text

In [11]:
text_lenght=[]
for i in df['text']:
    tokens = nltk.word_tokenize(i)
    text_lenght.append(len(tokens))


In [12]:
df['msge_w_count']=text_lenght

In [13]:
from collections import defaultdict
from collections import Counter
word_freq=[]
times_freq=[]
for i in df['text']:
    tokens = nltk.word_tokenize(i)
    fdist1=nltk.FreqDist(tokens)
    clean_w=[word for word in tokens if word.isalpha()]
    freq=dict()
    for word in clean_w:
        word=word.lower()
        freq[word]=freq.get(word,0)+1
        #w=[] 
        #w.append((max(freq, key=freq.get)))
        #d=defaultdict(int)
        for x in freq:
            freq[x] +=1
            wd=(max(freq.items(), key=lambda x:x[1]))
    word_freq.append(wd)
   


In [ ]:
wd

In [14]:
df['word_freq']=word_freq

In [15]:
df[['word_frq','times_frq']] = pd.DataFrame(df.word_freq.tolist(), index= df.index)


In [16]:
df.drop(['word_freq'], axis=1, inplace=True)

In [17]:
df

,user,text,file_content,reactions,channel,reply_users,replies_user,have_file,have_link,reactions_user,rep_time,date,time,msge_w_count,word_frq,times_frq
0,Sian Davies,<@U021UNQGZC4> has joined the channel,False,0,music,0,0,False,False,0,00:00:00,2021-05-12,09:50:00,8,has,5
1,Madini Liebscher,<@U0217P72YR5> has joined the channel,False,0,music,0,0,False,False,0,00:00:00,2021-05-12,09:50:00,8,has,5
2,miguel.simon,<@U021NHS652N> has joined the channel,False,0,music,0,0,False,False,0,00:00:00,2021-05-12,09:50:00,8,has,5
3,Jonas Tonndorff,<@U021TDD883V> has joined the channel,False,0,music,0,0,False,False,0,00:00:00,2021-05-13,09:52:00,8,has,5
4,Sian Davies,starting off our music recommendations - this ...,False,1,music,0,0,False,True,miguel.simon,00:00:00,2021-06-02,08:24:00,21,starting,15
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1822,Florian Titze,and you can only clone those that have this gr...,True,0,getting_started_tips,0,0,True,False,0,00:00:00,2021-06-01,07:34:00,12,and,12
1823,Uroš Petković,i see,False,0,getting_started_tips,0,0,False,False,0,00:00:00,2021-06-01,07:35:00,2,i,3
1824,Florian Titze,you can't only clone one folder - you could do...,False,0,getting_started_tips,0,0,False,False,0,00:00:00,2021-06-01,07:35:00,24,you,22
1825,Uroš Petković,so what do i do then? :slightly_smiling_face:,False,0,getting_started_tips,0,0,False,False,0,00:00:00,2021-06-01,07:35:00,10,so,7


In [ ]:
userReac=df.groupby(['user'])['reactions'].count()

In [ ]:
userReac.to_csv('userReac.csv',index=False)

In [ ]:
df_user.unstack().plot(figsize=(20,15))

In [21]:
df

,user,text,file_content,reactions,channel,reply_users,replies_user,have_file,have_link,reactions_user,rep_time,date,time,msge_w_count,word_frq,times_frq
0,Sian Davies,<@U021UNQGZC4> has joined the channel,False,0,music,0,0,False,False,0,00:00:00,2021-05-12,09:50:00,8,has,5
1,Madini Liebscher,<@U0217P72YR5> has joined the channel,False,0,music,0,0,False,False,0,00:00:00,2021-05-12,09:50:00,8,has,5
2,miguel.simon,<@U021NHS652N> has joined the channel,False,0,music,0,0,False,False,0,00:00:00,2021-05-12,09:50:00,8,has,5
3,Jonas Tonndorff,<@U021TDD883V> has joined the channel,False,0,music,0,0,False,False,0,00:00:00,2021-05-13,09:52:00,8,has,5
4,Sian Davies,starting off our music recommendations - this ...,False,1,music,0,0,False,True,miguel.simon,00:00:00,2021-06-02,08:24:00,21,starting,15
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1822,Florian Titze,and you can only clone those that have this gr...,True,0,getting_started_tips,0,0,True,False,0,00:00:00,2021-06-01,07:34:00,12,and,12
1823,Uroš Petković,i see,False,0,getting_started_tips,0,0,False,False,0,00:00:00,2021-06-01,07:35:00,2,i,3
1824,Florian Titze,you can't only clone one folder - you could do...,False,0,getting_started_tips,0,0,False,False,0,00:00:00,2021-06-01,07:35:00,24,you,22
1825,Uroš Petković,so what do i do then? :slightly_smiling_face:,False,0,getting_started_tips,0,0,False,False,0,00:00:00,2021-06-01,07:35:00,10,so,7


In [ ]:
df_cuser=df.groupby(['channel','user']).count()

In [ ]:
df.to_csv('Final_data.csv',index=False)

In [ ]:
df.shape

In [ ]:
us_rep=df.groupby(['channel','user'])['replies_user'].count()

In [ ]:
us_rep

In [ ]:
df['user']

In [36]:
hlink=df.groupby(['have_link','text']).count()

In [37]:
hlink

user  \
have_link text                                                       
False                                                           77   
          #TeamCoriander :mechanical_arm:                        1   
          (Change “google-chrome” for the program you wan...     1   
          (THANK YOU Anna) :pray:                                1   
          (cough) tableau might be easier for your desire...     1   
...                                                            ...   
True      well this is a depressing use of machine learni...     1   
          yes everything is here <https://www.notion.so/i...     1   
          you can also check out this curated list <https...     1   
          you can also get extensions for your browser wh...     1   
          “...com­pa­nies are ramp­ing up re­quire­ments ...     1   

                                                              file_content  \
have_link text                                                               
False                                                                   77   
          #TeamCoriander :mechanical_arm:                                1   
          (Change “google-chrome” for the program you wan...             1   
          (THANK YOU Anna) :pray:                                        1   
          (cough) tableau might be easier for your desire...             1   
...                                                                    ...   
True      well this is a depressing use of machine learni...             1   
          yes everything is here <https://www.notion.so/i...             1   
          you can also check out this curated list <https...             1   
          you can also get extensions for your browser wh...             1   
          “...com­pa­nies are ramp­ing up re­quire­ments ...             1   

                                                              reactions  \
have_link text                                                            
False                                                                77   
          #TeamCoriander :mechanical_arm:                             1   
          (Change “google-chrome” for the program you wan...          1   
          (THANK YOU Anna) :pray:                                     1   
          (cough) tableau might be easier for your desire...          1   
...                                                                 ...   
True      well this is a depressing use of machine learni...          1   
          yes everything is here <https://www.notion.so/i...          1   
          you can also check out this curated list <https...          1   
          you can also get extensions for your browser wh...          1   
          “...com­pa­nies are ramp­ing up re­quire­ments ...          1   

                                                              channel  \
have_link text                                                          
False                                                              77   
          #TeamCoriander :mechanical_arm:                           1   
          (Change “google-chrome” for the program you wan...        1   
          (THANK YOU Anna) :pray:                                   1   
          (cough) tableau might be easier for your desire...        1   
...                                                               ...   
True      well this is a depressing use of machine learni...        1   
          yes everything is here <https://www.notion.so/i...        1   
          you can also check out this curated list <https...        1   
          you can also get extensions for your browser wh...        1   
          “...com­pa­nies are ramp­ing up re­quire­ments ...        1   

                                                              reply_users  \
have_link text                                                              
False                                                   